# Almost pairs trading

This little exercise goes back to an idea by Stephen Boyd:

The simulator should always be completely agnostic as to the trading policy.
You should even demonstrate this with silly policies.
Like here’s one:  Each day choose names from the universe at random.
Buy one (say 0.1 of your portfolio wealth) and short one the same amount.
Not a good strategy, but a valid one.
Of course the simulate will terminate if you go bust (which seems likely).

In [6]:
from pathlib import Path

import numpy as np
import pandas as pd
from loguru import logger

from cvx.simulator.portfolio import build_portfolio
from cvx.simulator.trading_costs import LinearCostModel

pd.options.plotting.backend = "plotly"

In [7]:
logger.info("Load prices")
prices = pd.read_csv(Path("data") / "stock_prices.csv", index_col=0, parse_dates=True, header=0).ffill()

# Let's pay a fee of 10 bps every time
logger.info("Build trading cost model")
linearCostModel = LinearCostModel(name="LinearCostModel cost model", factor=0.0000)

logger.info("Build portfolio")
portfolio = build_portfolio(prices=prices, initial_cash=1e6, trading_cost_model=linearCostModel)

for before, now, state in portfolio:
    assert state.nav > 0, "Game over"

    # pick two assets at random
    pair = np.random.choice(portfolio.assets, 2, replace=False)
    
    # compute the pair
    stocks = pd.Series(index=portfolio.assets, data=0.0)
    stocks[pair] = [state.nav, -state.nav] / state.prices[pair].values

    portfolio[now] = 0.1*stocks

2023-05-08 10:25:39.466 | INFO     | __main__:<module>:1 - Load prices
2023-05-08 10:25:39.472 | INFO     | __main__:<module>:5 - Build trading cost model
2023-05-08 10:25:39.473 | INFO     | __main__:<module>:8 - Build portfolio


In [8]:
portfolio.nav.plot()